In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import signal
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from useful_func import *
import os
import seaborn as sns

In [ ]:
fs = 40

df = pd.read_csv('info_events.csv', index_col=0)
df['Picked arrival'] = pd.to_datetime(df['Picked arrival'])
df['file_start'] = pd.to_datetime(df['file_start'])
diff = (df['Picked arrival']- df['file_start']) #temps entre le debut de l'enregistrement et l'arrivée du séisme.
df['diff'] = diff.dt.total_seconds()
df = df.sort_values(by='Picked arrival')


In [ ]:
all_files = []
for dossier_actuel, sous_dossiers, fichiers in os.walk('data\DonneesB23'):
    if "_MACOSX" not in dossier_actuel:
        for fichier in fichiers:
            if '.DS_Store' not in fichier:
                path = os.path.join(dossier_actuel, fichier)
                y = np.fromfile(path, dtype=np.int32)
                if len(y)/fs > 2*60*60 :
                    all_files.append(path)
del y
noisy_files = []
known_files = list(df['file'].unique())
for file in all_files :
    if file not in known_files :
        noisy_files.append(file)

In [ ]:
def get_positives(files, events_time, demi_duree, n, fs=40, rd = True):

    signal_tronque = []
    signals = [ np.fromfile(file, dtype=np.int32) for file in files]

    i = 0
    while len(signal_tronque) < n :
        shift = np.random.normal(0, demi_duree/100, 1)[0]
        if not rd :
            shift = 0
        a = int(events_time[i] + shift - demi_duree)
        b = int(events_time[i] + shift + demi_duree)

        if a > 0 and b < len(signals[i])/fs :
            y = signals[i][ a*fs : b*fs ]
            signal_tronque.append(y)

        i = (i + 1) % len(files)

    return signal_tronque
        


def get_negatives(noisy_files, duree, nb_0, fs=40):
    signals = [ np.fromfile(file, dtype=np.int32) for file in noisy_files]

    X_0 = []
    i = 0
    while len(X_0) < nb_0 :
        T = int(len(signals[i])/fs)
        t = np.random.randint(T - duree)
        y = signals[i][t*fs : t*fs + duree*fs]
        X_0.append(y)
            
        i = (i + 1) % len(noisy_files)
    return X_0

In [ ]:
demi_duree = 60*60 

n = 500

#on enlève les séismes * et S1
mask = (df['Tag']!='S1') & (df['Tag']!='*') 

#20% pour le jeu de test, ne pas séparer après get_positive car le jeu de test sera biaisé
mask_t = np.random.randint(0, 10, len(mask)) < 8
files = df[ mask & mask_t]['file'].values
events_time = df[ mask & mask_t]['diff'].to_numpy(np.int32)


signal_tronque =  get_positives(files, events_time, demi_duree, n)

print(n, 'positives')

nb_0 = int(n*2)

X_0 = get_negatives(noisy_files, 2*demi_duree, nb_0)

print(nb_0, 'negatives')

y_0 = np.zeros(len(X_0))
y_1 = np.ones(len(signal_tronque))

X = np.concatenate((X_0, signal_tronque))
y = np.concatenate((y_0, y_1))


del X_0, y_0, y_1, signal_tronque


X = np.array(list(map(get_mov_rms, map(bp,  signal.decimate(X, 100)))))
print(X.shape)

In [ ]:

rd = np.random.permutation(len(X))
X_train = X[rd]
y_train = y[rd]
model = RandomForestClassifier(n_estimators=50, max_depth=20, criterion='gini', class_weight='balanced_subsample', verbose=0, n_jobs=-1, max_features='sqrt')
model.fit(X_train, y_train)

In [ ]:
files = df[ mask & (1-mask_t)]['file'].values
events_time = df[ mask & (1-mask_t)]['diff'].to_numpy(np.int32)


signal_tronque =  get_positives(files, events_time, demi_duree, len(files), rd = False)

test = np.array(list(map(get_mov_rms, map(bp,  signal.decimate(signal_tronque, 100)))))
res = model.predict(test)
test_0 = get_negatives(noisy_files, 2*demi_duree, 3*len(files))
test_0 = np.array(list(map(get_mov_rms, map(bp,  signal.decimate(test_0, 100)))))
res_0 = model.predict(test_0)

In [ ]:
tp = res
tn =  (1-res_0)
fp = res_0
fn = (1-res)
print(f"tp: {np.sum(tp)}")
print(f"tn: {np.sum(tn)}")
print(f"fp: {np.sum(fp)}")
print(f"fn: {np.sum(fn)}")
acc = (np.sum(tp)+np.sum(tn))/(np.sum(tp)+np.sum(tn)+np.sum(fp)+np.sum(fn))
print(f"accuracy: {acc}")
precision = np.sum(tp)/(np.sum(tp)+np.sum(fp))
recall = np.sum(tp)/(np.sum(tp)+np.sum(fn))
f1 = 2*precision*recall/(precision+recall)
print(f"precision: {round(precision,2)}")
print(f"recall: {round(recall,2)}")
print(f"f1 score: {round(f1, 2)}")
p0 = (np.sum(tp)+np.sum(tn))/(np.sum(tp)+np.sum(tn)+np.sum(fp)+np.sum(fn))
pe = ((np.sum(tp)+np.sum(fp))*(np.sum(tp)+np.sum(fn))+(np.sum(fn)+np.sum(tn))*(np.sum(fp)+np.sum(tn)))/((np.sum(tp)+np.sum(tn)+np.sum(fp)+np.sum(fn))**2)
kappa = (p0-pe)/(1-pe)
print(f"kappa: {round(kappa, 2)}")

In [ ]:
conf_matrix = np.array([[sum(tn), sum(fp)],
                        [sum(fn), sum(tp)]])

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g', 
            xticklabels=['Prédit négatif', 'Prédit positif'],
            yticklabels=['Vrai négatif', 'Vrai positif'],
            annot_kws={"fontsize": 40})  
plt.xlabel('Valeurs prédites', fontsize=12) 
plt.ylabel('Valeurs réelles', fontsize=12)  
plt.title('Matrice de confusion', fontsize=20) 
plt.show()

In [ ]:
#ne regarde quasiment que l'arrivée de l'onde S
plt.plot(model.feature_importances_)

# To visualise the results

In [ ]:
i = 0

In [ ]:
duree = 2*demi_duree
file = df['file'].iloc[i]
y = np.fromfile(file, dtype=np.int32)
T = len(y) 
intervalle = 60*60*fs 
nb_echant = int(T/intervalle)
print(nb_echant, 'samples to test')
X_t = []
for j in range(nb_echant) :
    if len(y[intervalle*j : intervalle*j + (duree)*fs]) == (duree)*fs :
        X_t.append(y[intervalle*j : intervalle*j + (duree)*fs])


X_t = np.array(X_t)
print(X_t.shape)

X_t = list(map(get_mov_rms, map(bp, signal.decimate(X_t, 100))))

predictions = model.predict(X_t)
indices = np.where(predictions == 1)[0]
print(len(indices), 'events detected')
y_d = bp(signal.decimate(y, 100))
plt.figure(figsize=(20, 7))
plt.plot(y_d)

for j in indices :
    plt.axvline(intervalle*j/100+(duree//2)*fs/100, color='r', linestyle='--', label='detected event')
plt.axvline(df['diff'].iloc[i]*fs/100, color='g', linestyle='--', label='known event')

plt.xlabel('Temps (s)')
plt.ylabel('Amplitude')
plt.title(f'seismic event detection on {i}th file')
plt.legend()
plt.show()

i = (i+1)%len(df['file'])

# If the model is strong

In [ ]:
import pickle
with open('big_random_forest.pkl', 'wb') as f:
    pickle.dump(model, f)